In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from torch import optim
from sklearn import preprocessing
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import median_absolute_error

In [3]:
import matplotlib

In [4]:
pd = nb_setup.setup_pandas(escape_latex = False)

NameError: name 'nb_setup' is not defined

### Daten einlesen

In [4]:
rawdat = pd.read_csv('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/rawdat.csv', sep = ';')

In [5]:
rawdat = rawdat[['HomeTeam','AwayTeam','FTHG','FTAG','FTR','HTR','HS','AS','HST','AST']]

In [6]:
rawdat.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTR,HS,AS,HST,AST
0,Aston Villa,Wigan,0,2,A,A,11,14,5,7
1,Blackburn,Man City,0,2,A,A,17,8,9,5
2,Bolton,Sunderland,0,1,A,A,11,20,3,13
3,Chelsea,Hull,2,1,H,D,26,7,12,3
4,Everton,Arsenal,1,6,A,A,8,15,5,9


### Daten verarbeiten und kodieren

In [7]:
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
#Test zum pushen

In [8]:
# Methode setzt Punkte für Heim und Auswärtsteam und erstellt neue Spalten
def windrawloseHomeAway(data):
    data = data[['HomeTeam', 'AwayTeam', 'FTR']]
    data['FTRH']= pd.Series(np.zeros(len(data['FTR'])))
    data['FTRA']= pd.Series(np.zeros(len(data['FTR'])))
    i = 0
    ftrH = 0
    ftrA = 0
    for row in data.itertuples():
        ftr = data.iloc[i]['FTR']
        if (ftr == 'H'):
            ftrH = 3
            ftrA = 0
        if (ftr == 'A'):
            ftrH = 0
            ftrA = 3
        if (ftr == 'D'):
            ftrH = 1
            ftrA = 1
        data.set_value(i,'FTRH', ftrH)
        data.set_value(i, 'FTRA', ftrA)
        i = i+1
    return data[['HomeTeam', 'AwayTeam', 'FTRH', 'FTRA','FTR']]
    

In [9]:
new_df = windrawloseHomeAway(rawdat)

/Users/fabian-maltemoller/Developer/Anaconda/anaconda3/envs/KiTipico/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/Users/fabian-maltemoller/Developer/Anaconda/anaconda3/envs/KiTipico/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [10]:
homeP = new_df['FTRH']
awayP = new_df['FTRA']

In [11]:
# Dimensionalität richten
homeP= np.resize(homeP,(homeP.size, 1))
awayP = np.resize(awayP,(awayP.size,1))
print(homeP.shape)
print(awayP.shape)

(3800, 1)
(3800, 1)


#### Punkte zwischen 0 und 1 skalieren

In [12]:
min_max_scaler = preprocessing.MinMaxScaler()
homeP_scaled = min_max_scaler.fit_transform(homeP)

In [13]:
min_max_scaler = preprocessing.MinMaxScaler()
awayP_scaled = min_max_scaler.fit_transform(awayP)

In [14]:
df_scale_homeP = pd.DataFrame(homeP_scaled)
df_scale_awayP = pd.DataFrame(awayP_scaled)
new_df['SPH'] = df_scale_homeP
new_df['SPA'] = df_scale_awayP
new_df.head()

,HomeTeam,AwayTeam,FTRH,FTRA,FTR,SPH,SPA
0,Aston Villa,Wigan,0.0,3.0,A,0.0,1.0
1,Blackburn,Man City,0.0,3.0,A,0.0,1.0
2,Bolton,Sunderland,0.0,3.0,A,0.0,1.0
3,Chelsea,Hull,3.0,0.0,H,1.0,0.0
4,Everton,Arsenal,0.0,3.0,A,0.0,1.0


In [15]:
min_max_scaler.scale_

array([0.33333333])

In [16]:
new_df['SPH'][0]/min_max_scaler.scale_[0] == new_df['FTRH'][0]

True

In [17]:
new_df.head()

,HomeTeam,AwayTeam,FTRH,FTRA,FTR,SPH,SPA
0,Aston Villa,Wigan,0.0,3.0,A,0.0,1.0
1,Blackburn,Man City,0.0,3.0,A,0.0,1.0
2,Bolton,Sunderland,0.0,3.0,A,0.0,1.0
3,Chelsea,Hull,3.0,0.0,H,1.0,0.0
4,Everton,Arsenal,0.0,3.0,A,0.0,1.0


### Input und Targets generieren

In [18]:
input = []
target= []
for row in new_df.itertuples():
    inputdf = pd.DataFrame()
    inputdf_1 = pd.DataFrame()
    homeTeam = row.HomeTeam
    awayTeam = row.AwayTeam
    index = row.Index
    df_home = new_df[new_df['HomeTeam']==homeTeam]
    df_home = df_home.truncate(after= index)
    df_home.drop(index, inplace=True)
    inputdf['H'] = df_home.tail(20)['SPH']
    if len(inputdf.index) == 20:
        df_away = new_df[new_df['AwayTeam']==awayTeam]
        df_away = df_away.truncate(after= index)
        df_away.drop(index, inplace=True)
        inputdf_1['A']= df_away.tail(20)['SPA']
        if len(inputdf_1.index) == 20:
            inputdf['A']= inputdf_1['A'].values
            inputlist = inputdf.values.tolist()
            target_df = row.FTR
            input.append(inputlist)
            target.append(target_df)

In [19]:
input[0]

[[0.0, 1.0],
 [0.3333333333333333, 0.0],
 [1.0, 0.0],
 [1.0, 1.0],
 [1.0, 0.3333333333333333],
 [1.0, 1.0],
 [0.3333333333333333, 0.0],
 [0.3333333333333333, 1.0],
 [0.0, 0.3333333333333333],
 [0.3333333333333333, 1.0],
 [1.0, 1.0],
 [1.0, 0.3333333333333333],
 [1.0, 0.0],
 [1.0, 1.0],
 [0.3333333333333333, 1.0],
 [1.0, 0.3333333333333333],
 [0.3333333333333333, 0.0],
 [0.0, 0.0],
 [1.0, 0.0],
 [1.0, 0.3333333333333333]]

In [20]:
len(input) == len(target)

True

In [21]:
len(input)

2789

In [22]:
for input_set in input:
    for time_step in input_set:
        dif = time_step[0]-time_step[1]
        del time_step[1]
        time_step[0] = dif

In [23]:
input[0]

[[-1.0],
 [0.3333333333333333],
 [1.0],
 [0.0],
 [0.6666666666666667],
 [0.0],
 [0.3333333333333333],
 [-0.6666666666666667],
 [-0.3333333333333333],
 [-0.6666666666666667],
 [0.0],
 [0.6666666666666667],
 [1.0],
 [0.0],
 [-0.6666666666666667],
 [0.6666666666666667],
 [0.3333333333333333],
 [0.0],
 [1.0],
 [0.6666666666666667]]

In [24]:
min_max_scaler_dif = preprocessing.MinMaxScaler()
min_max_scaler_dif.fit_transform(input[0])

array([[0.        ],
       [0.66666667],
       [1.        ],
       [0.5       ],
       [0.83333333],
       [0.5       ],
       [0.66666667],
       [0.16666667],
       [0.33333333],
       [0.16666667],
       [0.5       ],
       [0.83333333],
       [1.        ],
       [0.5       ],
       [0.16666667],
       [0.83333333],
       [0.66666667],
       [0.5       ],
       [1.        ],
       [0.83333333]])

In [25]:
input[3]

[[0.0],
 [1.0],
 [0.0],
 [0.3333333333333333],
 [0.0],
 [1.0],
 [0.3333333333333333],
 [0.6666666666666667],
 [0.6666666666666667],
 [-0.3333333333333333],
 [-0.6666666666666667],
 [0.3333333333333333],
 [0.3333333333333333],
 [0.6666666666666667],
 [-0.6666666666666667],
 [0.0],
 [0.0],
 [0.0],
 [-0.3333333333333333],
 [1.0]]

In [26]:
for i in range(len(input)):
    input[i] = min_max_scaler_dif.transform(input[i]).tolist()

In [27]:
# Aus den Targets einen Skalarwert machen!
target_scalar = []
for result in target:
    if result == 'H':
        data = [1.]
    elif result == 'D':
        data = [0.5]
    else:
        data = [0.]
    target_scalar.append(data)

In [28]:
len(input) == len(target_scalar)

True

In [29]:
type(target_scalar[0])

list

In [30]:
target_scalar[3]

[1.0]

In [31]:
input[3]

[[0.5],
 [1.0],
 [0.5],
 [0.6666666666666666],
 [0.5],
 [1.0],
 [0.6666666666666666],
 [0.8333333333333334],
 [0.8333333333333334],
 [0.33333333333333337],
 [0.16666666666666663],
 [0.6666666666666666],
 [0.6666666666666666],
 [0.8333333333333334],
 [0.16666666666666663],
 [0.5],
 [0.5],
 [0.5],
 [0.33333333333333337],
 [1.0]]

In [32]:
correct= 0
for i in range(len(input)):
    if(len(input[i]) == 20):
        correct +=1

In [33]:
correct/len(input)

1.0

In [34]:
#for df in input:
#    df = df.to_numpy()

### Daten in Test und Trainingsdaten teilen

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
input_train, input_test, target_train, target_test = train_test_split(input, target_scalar, test_size=0.20) 

### Datensatz aus Input und Target erstellen

In [37]:
class TempDataset(Dataset):
    def __init__(self, input, target):
        self.input=input
        self.target=target
        
    def __len__(self):
        return len(self.target)
    
    def __getitem__(self, index):
        x = self.input[index]
        y = self.target[index]
        
        return x, y

In [38]:
#Trainingsdatensatz erstellen
training_set = TempDataset(input_train, target_train)

In [39]:
len(training_set)/len(target_scalar)

0.7999282897095733

In [40]:
len(input_train)

2231

In [41]:
del input_train[2230]
del target_train[2230]

In [42]:
for input_sets in input_train:
    if len(input_sets) != 20:
        print(input_set)

In [43]:
for i in range(len(input_train)):
    if not input_train[i]:
        print('i ='+i)
    for k in range(len(input_train[i])):
        if not input_train[i][k]:
            print('k ='+k+'     i='+i)

In [44]:
# Trainingsdatensatz in richtige Form bringen
def mycollate(batch):
    input = torch.Tensor([item[0] for item in batch]).view(20,10,1)
    target= torch.Tensor([item[1] for item in batch]).view(10,1)
    return input, target

In [45]:
params = {'batch_size' : 10,
         'shuffle' : False,
         'num_workers' : 4,
         'collate_fn' : mycollate}

In [46]:
training_generator = DataLoader(training_set, **params)

In [47]:
x,y = next(iter(training_generator))
x.size()

torch.Size([20, 10, 1])

In [48]:
x

tensor([[[0.0000],
         [0.1667],
         [1.0000],
         [0.1667],
         [0.8333],
         [0.5000],
         [0.5000],
         [1.0000],
         [0.5000],
         [0.5000]],

        [[0.5000],
         [0.5000],
         [0.8333],
         [0.8333],
         [1.0000],
         [0.6667],
         [0.5000],
         [0.5000],
         [0.3333],
         [0.5000]],

        [[1.0000],
         [0.5000],
         [0.5000],
         [0.0000],
         [0.5000],
         [0.5000],
         [0.1667],
         [1.0000],
         [0.5000],
         [0.0000]],

        [[1.0000],
         [0.5000],
         [0.8333],
         [0.8333],
         [1.0000],
         [0.0000],
         [0.5000],
         [0.5000],
         [0.8333],
         [0.5000]],

        [[0.6667],
         [0.5000],
         [0.0000],
         [0.5000],
         [0.6667],
         [1.0000],
         [0.0000],
         [1.0000],
         [0.5000],
         [0.0000]],

        [[0.8333],
         [0.5000],
  

In [49]:
y.size()

torch.Size([10, 1])

## LSTM-Netz errichten

In [50]:
BATCH_SIZE = params['batch_size']
NUM_LAYERS = 3

class LSTMnn(nn.Module):
    
    def __init__(self , input_size , hidden_size , output_size , 
                 batch_size = BATCH_SIZE , num_layers = NUM_LAYERS ) :
        super(LSTMnn, self).__init__()
        self.batch_size = batch_size
        # creation LSTM LAYER
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first = False)
        self.hidden = self.init_hidden()
        #Creation Linear Layer
        self.output_size = output_size
        self.linear = nn.Linear(self.hidden_size, self.output_size)
        
    def init_hidden(self): # random Weights will be generated
        return (torch.rand(self.num_layers, self.batch_size, self.hidden_size),
               torch.rand(self.num_layers, self.batch_size, self.hidden_size))
    
    def forward(self, input_seqence):
        out, _ = self.lstm(input_seqence, self.hidden)
        output = torch.sigmoid(self.linear(out[-1]))
        return output

In [51]:
# 
lstm = LSTMnn(input_size=1, hidden_size=20, output_size=1, batch_size= BATCH_SIZE, num_layers=NUM_LAYERS)

In [52]:
# Fehlerfinktion und Lernfunktion 
loss_function = nn.MSELoss()
optimizer = optim.Adam(lstm.parameters(), lr =0.03)

In [53]:
loss_memory= []
for epoch in range(5):
    for local_batch, local_labels in training_generator:
        lstm.zero_grad()
        if local_batch.size()[1] == BATCH_SIZE:
            out = lstm(local_batch)
            loss = loss_function(out, local_labels)
            loss_memory.append(loss)
            loss.backward(retain_graph = True)
            optimizer.step()
    print('Fehler letzter Batch: {}'.format(loss.item()))

Fehler letzter Batch: 0.19034835696220398
Fehler letzter Batch: 0.19035951793193817
Fehler letzter Batch: 0.1903536468744278
Fehler letzter Batch: 0.19035561382770538
Fehler letzter Batch: 0.19035489857196808


In [54]:
lstm(x)

tensor([[0.5828],
        [0.5828],
        [0.5828],
        [0.5828],
        [0.5828],
        [0.5828],
        [0.5828],
        [0.5828],
        [0.5828],
        [0.5828]], grad_fn=<SigmoidBackward>)

In [55]:
from ipypublish import nb_setup

ModuleNotFoundError: No module named 'ipypublish'

In [56]:
y = [item.item() for item in loss_memory]
x = range(len(loss_memory))
plt = nb_setup.setup_matplotlib(output =('png','png'))
plt.plot(x, y, label = 'Fehlerentwicklung')
plt.ylabel(r'Fehler')
plt.xlabel(r'Anzahl Trainingsbatches')
plt.legend();

NameError: name 'nb_setup' is not defined

In [63]:
x

tensor([[[0.3333, 1.0000],
         [0.0000, 0.3333],
         [1.0000, 0.0000],
         [1.0000, 1.0000],
         [0.0000, 0.0000],
         [0.0000, 1.0000],
         [1.0000, 1.0000],
         [0.0000, 0.0000],
         [0.0000, 0.0000],
         [1.0000, 1.0000],
         [0.0000, 0.0000],
         [0.3333, 0.0000],
         [0.0000, 0.0000],
         [1.0000, 0.0000],
         [1.0000, 0.0000],
         [0.0000, 1.0000]],

        [[0.3333, 0.0000],
         [1.0000, 0.0000],
         [0.0000, 0.3333],
         [0.0000, 1.0000],
         [0.3333, 1.0000],
         [0.0000, 0.3333],
         [1.0000, 1.0000],
         [0.0000, 1.0000],
         [0.0000, 0.3333],
         [0.3333, 0.3333],
         [1.0000, 1.0000],
         [1.0000, 0.0000],
         [1.0000, 1.0000],
         [0.0000, 0.3333],
         [0.3333, 0.0000],
         [1.0000, 0.0000]],

        [[1.0000, 0.0000],
         [1.0000, 0.0000],
         [0.3333, 0.0000],
         [0.3333, 1.0000],
         [1.0000, 1.0000

In [64]:
y

tensor([[[0., 1., 0.],
         [0., 0., 1.],
         [0., 1., 0.],
         [0., 1., 0.],
         [1., 0., 0.],
         [0., 1., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.],
         [1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.],
         [0., 1., 0.],
         [0., 1., 0.]]])